Below code comes from https://github.com/jnyborg/timematch

Trial Output

In [1]:
import torch
import numpy as np
import torch.nn as nn
import json
from torchvision.transforms import transforms
from utils import RandomSamplePixels, Normalize, ToTensor, PixelSetData,\
    split_dict_train_test, pad_sequences_collate_fn, LinearLayer, PixelSetEncoder,\
        PositionalEncoding, TimeSeriesTransformer, SimpleMLP
from collections import Counter
from torch.utils.data import DataLoader

LinearLayer

In [2]:
# pixels_tmp = pixel_dataset[1901]['pixels'].permute(0, 2, 1)
pixels_tmp = torch.randn(52, 10, 32).permute(0, 2, 1)
print("input", pixels_tmp.shape)

ll = LinearLayer(10, 64)

print(ll(pixels_tmp).shape)

input torch.Size([52, 32, 10])
torch.Size([52, 32, 64])


Δοκιμή του LinearLayer με τυχαία διανύσματα όμως ΧΩΡΙΣ μάσκες

In [3]:
# mlp1_dim=[10, 32, 64]
mlp1_dim=[10, 32]
mlp2_dim=[64, 128]

layers = []
for i in range(len(mlp1_dim) - 1):
    layers.append(LinearLayer(mlp1_dim[i], mlp1_dim[i + 1]))
mlp1 = nn.Sequential(*layers)
print(mlp1)

layers = []
for i in range(len(mlp2_dim) - 1):
    layers.append(LinearLayer(mlp2_dim[i], mlp2_dim[i + 1]))
mlp2 = nn.Sequential(*layers)
print(mlp2)

# out = pixel_dataset[1901]['pixels'].unsqueeze(0)
out = torch.randn(52, 10, 32).unsqueeze(0)
print(out.shape) # torch.Size([1, 52, 10, 32])

batch, temp = out.shape[:2]
# print(batch, temp) # 1 52

out = out.view(batch * temp, *out.shape[2:]).transpose(1, 2)  # (B*T, S, C)
print(out.shape) # torch.Size([52, 32, 10])

out = mlp1(out).transpose(1, 2)
print(out.shape) # torch.Size([52, 32, 32])

out = torch.cat(
            [out.mean(dim=-1), out.std(dim=-1)],
            dim=1)
print(out.shape) # torch.Size([52, 64])

out = mlp2(out)
print(out.shape) # torch.Size([52, 128])

out = out.view(batch, temp, -1)
print(out.shape) # torch.Size([1, 52, 128])

Sequential(
  (0): LinearLayer(
    (linear): Linear(in_features=10, out_features=32, bias=False)
    (norm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation): ReLU()
  )
)
Sequential(
  (0): LinearLayer(
    (linear): Linear(in_features=64, out_features=128, bias=False)
    (norm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation): ReLU()
  )
)
torch.Size([1, 52, 10, 32])
torch.Size([52, 32, 10])
torch.Size([52, 32, 32])
torch.Size([52, 64])
torch.Size([52, 128])
torch.Size([1, 52, 128])


## PSE Part

Οι extra χωρικές πληροφορίες δε χρειάζονται

Δημιουργία Dataset και Dataloaders για τη δοκιμή του PSE και του Transformer Encoder

In [4]:
# labels_200 will contain the labels with more than 200 occurrences
f_labels = open(r"Exercise4\timematch_data\denmark\32VNH\2017\meta\labels_cleaned.json")
labels_200 = json.load(f_labels)

print(len(labels_200), len(set(labels_200.values())))

count = 0
for lab in labels_200:
    print(lab, labels_200[lab])
    count += 1
    if count == 6:
        break

labels_200_counter = Counter(labels_200.values())

print()

for lab in labels_200_counter:
    print(lab, labels_200_counter[lab])

4255 7
0 corn
1 corn
2 corn
3 corn
4 corn
5 spring_barley

corn 275
spring_barley 1141
meadow 1013
winter_wheat 856
winter_rapeseed 301
winter_barley 352
winter_rye 317


Οι παρακάτω αντιστοιχίσεις είναι διαφορετικές από του dataset_creation αλλά δεν πειράζει

In [5]:
class_to_idx = {cls: idx for idx, cls in enumerate(labels_200_counter)}
for key, val in class_to_idx.items():
    print(key, val)

print()

# train_labels, val_labels = split_dict_train_test(labels_200, test_size=0.2)
train_labels, val_labels, test_labels = split_dict_train_test(labels_200, test_size=0.007, val_size=0.2)

print(len(labels_200))
print(len(train_labels))
print(len(val_labels))
print(len(test_labels))

corn 0
spring_barley 1
meadow 2
winter_wheat 3
winter_rapeseed 4
winter_barley 5
winter_rye 6

4255
3371
849
35


In [6]:
train_transform = transforms.Compose([RandomSamplePixels(32), Normalize(), ToTensor()])

test_transform = transforms.Compose([Normalize(), ToTensor()])

train_dataset = PixelSetData("Exercise4/timematch_data/denmark/32VNH/2017",
                             class_to_idx, train_labels,
                             train_transform)
print(len(train_dataset))
print()
val_dataset = PixelSetData("Exercise4/timematch_data/denmark/32VNH/2017",
                             class_to_idx, val_labels,
                             test_transform)
print(len(val_dataset))

3371

849


In [7]:
train_dloader = DataLoader(
    train_dataset, batch_size=64, shuffle=True,
    collate_fn=pad_sequences_collate_fn,
    # num_workers=4,
)
val_dloader = DataLoader(
    val_dataset, batch_size=8, shuffle=True,
    collate_fn=pad_sequences_collate_fn,
)

print(len(train_dloader))
print(len(val_dloader))

for batch in train_dloader:
    print(batch[0].shape)
    print(batch[1].shape)
    print(batch[1])
    print(batch[2].shape)
    print(np.unique(batch[2]))
    print()
    break

for batch in val_dloader:
    print(batch[0].shape)
    print(batch[1].shape)
    print(batch[1])
    print(batch[2].shape)
    print(np.unique(batch[2]))
    print()
    break

53
107
torch.Size([64, 52, 10, 32])
torch.Size([64])
tensor([2, 3, 1, 4, 2, 1, 2, 3, 3, 2, 3, 4, 1, 6, 2, 2, 6, 1, 6, 2, 2, 6, 3, 0,
        6, 0, 1, 3, 0, 1, 6, 3, 2, 5, 5, 1, 1, 4, 3, 1, 5, 2, 1, 3, 2, 2, 3, 1,
        3, 5, 5, 0, 2, 1, 1, 6, 4, 2, 3, 1, 1, 1, 6, 3])
torch.Size([64, 52, 32])
[ True]

torch.Size([8, 52, 10, 3526])
torch.Size([8])
tensor([6, 1, 2, 4, 2, 1, 2, 4])
torch.Size([8, 52, 3526])
[False  True]



In [8]:
pixel_set_encoder = PixelSetEncoder(10, mlp1=[10, 32],
                                    pooling="mean_std",
                                    mlp2=[64, 128])
print(pixel_set_encoder)
print()

for batch in train_dloader:
    input_tensor = batch[0]
    labels = batch[1]
    mask = batch[2]

    print(input_tensor.shape)
    print(labels.shape)
    print(mask.shape)

    break

print()

out = pixel_set_encoder(input_tensor, mask)
print(out.shape)

print()

print(np.unique(out.detach().numpy()))

PixelSetEncoder(
  (mlp1): Sequential(
    (0): LinearLayer(
      (linear): Linear(in_features=10, out_features=32, bias=False)
      (norm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
    )
  )
  (mlp2): Sequential(
    (0): LinearLayer(
      (linear): Linear(in_features=64, out_features=128, bias=False)
      (norm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
    )
  )
)

torch.Size([64, 52, 10, 32])
torch.Size([64])
torch.Size([64, 52, 32])

torch.Size([64, 52, 128])

[0.0000000e+00 1.0728836e-05 1.5757978e-05 ... 9.5031271e+00 9.6956177e+00
 9.7326565e+00]


## Transformer Part

In [9]:
# can also check Tsironis
pe = PositionalEncoding(128)

input = out
out = pe(input)

print(out.shape)

torch.Size([64, 52, 128])


In [10]:
# time_series_transformer = TimeSeriesTransformer(feature_size=128)
time_series_transformer = TimeSeriesTransformer(d_model=128)
print(time_series_transformer)

# input = torch.randn(1, 52, 128)
input = out
out = time_series_transformer(input)
print("Output shape: ", out.shape)

TimeSeriesTransformer(
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (pos_emb): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
)
Output shape:  torch.Size([64, 128])


In [11]:
simple_mlp = SimpleMLP(input_dim=128, output_dim=7)
print(simple_mlp)

input = out
out = simple_mlp(input)

print(out.shape)

print(out)

SimpleMLP(
  (fc): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=7, bias=True)
  )
)
torch.Size([64, 7])
tensor([[-0.1746,  0.2504, -0.0499,  0.5450,  0.8937, -0.2919, -0.3470],
        [-0.1821,  0.2377,  0.1150,  0.5664,  0.9340, -0.3612, -0.2917],
        [-0.2063,  0.4163,  0.0055,  0.5068,  0.9041, -0.4562, -0.3397],
        [-0.1338,  0.2349, -0.1106,  0.6577,  0.9399, -0.3407, -0.4588],
        [-0.1590,  0.3552,  0.0870,  0.7250,  0.9227, -0.3058, -0.3813],
        [-0.1585,  0.3536,  0.1522,  0.6340,  0.8709, -0.3371, -0.2816],
        [-0.2002,  0.2721,  0.1608,  0.6488,  0.7744, -0.3228, -0.3664],
        [-0.2745,  0.4704, -0.2115,  0.7547,  0.8209, -0.3990, -0.3841],
        [-0.2035,  0.3238,  0.0669,  0.5581,  0.7886, -0.2964, -0.4057],
        [-0.1145,  0.2535, -0.0115,  0.6677,  0.8945, -0.2676, -0.4759],
        [-0.1480,  0.2993,  0.0103,  0.6005,  0.8956, -0.3908, -0.2354],
    

In [12]:
pixel_encoder = PixelSetEncoder(10, mlp1=[10, 32],
                                pooling="mean_std",
                                mlp2=[64, 128])

# transformer_encoder = TimeSeriesTransformer(feature_size=128)
transformer_encoder = TimeSeriesTransformer(d_model=128)

simple_mlp = SimpleMLP(input_dim=128, output_dim=7)

In [13]:
class CompleteModel(nn.Module):
    def __init__(self, pixel_encoder, transformer_encoder, simple_mlp):
        super(CompleteModel, self).__init__()
        self.pixel_encoder = pixel_encoder
        self.transformer_encoder = transformer_encoder
        self.simple_mlp = simple_mlp
    
    def forward(self, x, mask):
        x = self.pixel_encoder(x, mask)  # torch.Size([8, 52, 128])
        x = self.transformer_encoder(x)  # torch.Size([8, 128])
        x = self.simple_mlp(x) # torch.Size([8, 7])
        return x

model = CompleteModel(pixel_encoder, transformer_encoder, simple_mlp)

Δοκιμή Ολόκληρου Μοντέλου για είσοδο από Train Dataset με σταθερό πλήθος pixels = 32

In [14]:
train_dloader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    collate_fn=pad_sequences_collate_fn,
    # num_workers=4,
)

for batch in train_dloader:
    input_tensor = batch[0]
    labels = batch[1]
    mask = batch[2]

    print(input_tensor.shape)
    print(labels.shape)
    print(labels)
    print(mask.shape)
    print()

    break

out = model(input_tensor, mask)

print(out.shape)

print(out)

torch.Size([64, 52, 10, 32])
torch.Size([64])
tensor([0, 1, 1, 1, 5, 2, 1, 4, 3, 2, 1, 3, 2, 6, 1, 4, 5, 0, 0, 4, 2, 2, 5, 5,
        1, 3, 2, 5, 2, 1, 1, 1, 1, 3, 1, 5, 4, 0, 1, 1, 1, 5, 1, 1, 1, 1, 2, 6,
        3, 2, 3, 1, 2, 4, 1, 2, 1, 2, 1, 6, 0, 2, 1, 2])
torch.Size([64, 52, 32])

torch.Size([64, 7])
tensor([[ 0.0673, -0.2916,  0.0263,  0.4562, -0.2787,  0.5332, -0.3714],
        [ 0.2055, -0.4213,  0.1092,  0.4613, -0.2087,  0.5456, -0.4857],
        [ 0.1898, -0.3632, -0.0169,  0.2836, -0.1463,  0.5887, -0.4524],
        [ 0.0788, -0.2758,  0.1793,  0.2031, -0.2136,  0.4173, -0.1908],
        [ 0.1343, -0.4454, -0.0020,  0.3815, -0.2549,  0.5937, -0.3633],
        [ 0.1074, -0.4615,  0.0320,  0.2680, -0.1209,  0.5287, -0.2691],
        [ 0.1954, -0.3636,  0.0434,  0.3860, -0.2647,  0.5393, -0.4162],
        [ 0.1652, -0.5126, -0.1073,  0.3853, -0.4215,  0.5480, -0.2626],
        [ 0.0869, -0.2835,  0.0684,  0.4643, -0.3630,  0.4841, -0.4420],
        [ 0.2171, -0.5031,  0.0430

Δοκιμή Ολόκληρου Μοντέλου για είσοδο από Val Dataset με μεταβλητό πλήθος pixels

In [15]:
for batch in val_dloader:
    input_tensor = batch[0]
    labels = batch[1]
    mask = batch[2]

    print(input_tensor.shape)
    print(labels.shape)
    print(labels)
    print(mask.shape)
    print()

    break

out = model(input_tensor, mask)

print(out.shape)

print(out)

torch.Size([8, 52, 10, 984])
torch.Size([8])
tensor([2, 1, 1, 3, 0, 2, 1, 6])
torch.Size([8, 52, 984])

torch.Size([8, 7])
tensor([[ 4.2552e-02, -2.4886e-01,  1.3728e-03,  5.4222e-01, -3.4338e-01,
          3.8799e-01, -4.0346e-01],
        [ 2.3111e-01, -3.9134e-01,  2.2441e-05,  4.1829e-01, -1.2590e-01,
          5.0973e-01, -5.2978e-01],
        [ 7.5253e-02, -3.8004e-01,  1.1530e-01,  3.4886e-01, -2.7648e-01,
          4.5949e-01, -2.7855e-01],
        [ 2.0189e-01, -3.2991e-01,  9.9848e-02,  4.5252e-01, -2.3260e-01,
          5.3377e-01, -5.2884e-01],
        [ 7.2022e-02, -4.5869e-01, -1.4452e-01,  4.5378e-01, -2.5426e-01,
          4.3806e-01, -3.8581e-01],
        [ 1.3725e-01, -3.6623e-01,  1.0538e-01,  3.0821e-01, -3.3143e-01,
          4.0145e-01, -2.3071e-01],
        [ 1.2191e-01, -3.4449e-01,  9.2791e-02,  2.6776e-01, -3.2138e-01,
          4.7177e-01, -3.4768e-01],
        [ 1.1603e-01, -1.8740e-01,  7.6889e-02,  3.5534e-01, -3.4639e-01,
          4.8111e-01, -4.8802e-01